In [1]:
import pandas as pd
import numpy as np

In [2]:
# Paths for train/test/submission
TRAIN_PATH = 'train.csv'
TEST_PATH  = 'test.csv'

# Load
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)

In [5]:
# Drop irrelevant ID-type features
for df in [train, test]:
    if 'id' in df.columns:
        df.drop(columns=['id', 'hospital_number'], inplace=True, errors='ignore')

# Fix types
for col in train.select_dtypes(include='object'):
    train[col] = train[col].astype('category')
    test[col]  = test[col].astype('category')

test

,surgery,age,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,38.6,40.0,20.0,normal,normal,normal_pink,less_3_sec,mild_pain,...,distend_small,42.0,7.5,clear,2.3,no,0,0,0,no
1,yes,adult,38.2,112.0,48.0,cool,reduced,bright_pink,more_3_sec,depressed,...,distend_small,44.0,6.0,serosanguious,2.6,no,2208,0,0,yes
2,yes,adult,37.7,66.0,12.0,cool,normal,bright_red,less_3_sec,mild_pain,...,distend_small,31.5,6.0,cloudy,1.6,yes,2205,0,0,yes
3,no,adult,37.1,88.0,20.0,cool,reduced,pale_cyanotic,less_3_sec,depressed,...,distend_large,75.0,81.0,NaN,1.0,yes,1400,0,0,no
4,yes,adult,38.3,50.0,12.0,NaN,normal,bright_pink,less_3_sec,mild_pain,...,distend_small,37.0,6.8,cloudy,2.6,yes,2208,0,0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,no,adult,40.3,114.0,36.0,cool,reduced,normal_pink,more_3_sec,depressed,...,distend_large,57.0,8.1,serosanguious,4.5,yes,3205,0,0,yes
820,yes,adult,37.2,100.0,20.0,cool,reduced,pale_cyanotic,more_3_sec,extreme_pain,...,distend_small,50.0,66.0,serosanguious,2.0,yes,2209,0,0,no
821,yes,adult,39.2,132.0,12.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,...,NaN,53.0,7.6,serosanguious,4.5,yes,2205,0,0,no
822,no,adult,38.3,54.0,66.0,normal,normal,normal_pink,less_3_sec,mild_pain,...,NaN,49.0,8.6,clear,5.0,no,3111,0,0,yes
